In [1]:
import torch
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import sys
sys.path.append("../../")
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import argparse

In [2]:
parser = argparse.ArgumentParser(description='Sequence Modeling - (Permuted) Sequential MNIST')
parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                    help='batch size (default: 64)')
parser.add_argument('--cuda', action='store_false',
                    help='use CUDA (default: True)')
parser.add_argument('--dropout', type=float, default=0.05,
                    help='dropout applied to layers (default: 0.05)')
parser.add_argument('--clip', type=float, default=-1,
                    help='gradient clip, -1 means no clip (default: -1)')
parser.add_argument('--epochs', type=int, default=20,
                    help='upper epoch limit (default: 20)')
parser.add_argument('--ksize', type=int, default=7,
                    help='kernel size (default: 7)')
parser.add_argument('--levels', type=int, default=8,
                    help='# of levels (default: 8)')
parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                    help='report interval (default: 100')
parser.add_argument('--lr', type=float, default=2e-3,
                    help='initial learning rate (default: 2e-3)')
parser.add_argument('--optim', type=str, default='Adam',
                    help='optimizer to use (default: Adam)')
parser.add_argument('--nhid', type=int, default=25,
                    help='number of hidden units per layer (default: 25)')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed (default: 1111)')
parser.add_argument('--permute', action='store_true',
                    help='use permuted MNIST (default: false)')
args = parser.parse_args()

torch.manual_seed(args.seed)

usage: ipykernel_launcher.py [-h] [--batch_size N] [--cuda]
                             [--dropout DROPOUT] [--clip CLIP]
                             [--epochs EPOCHS] [--ksize KSIZE]
                             [--levels LEVELS] [--log-interval N] [--lr LR]
                             [--optim OPTIM] [--nhid NHID] [--seed SEED]
                             [--permute]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/rysul/Library/Jupyter/runtime/kernel-4e1d6b14-5258-4b4b-814b-3788c2127a39.json


SystemExit: 2

/Users/rysul/opt/anaconda3/envs/torchEnv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
def customOneHotEncoder(data):
    dataAdjust = data.ljust(200,'0')[:200] # padding if not of length and adjusting the data lenght to get a 200x39 input matrix
    # define universe of possible input values
    alphabet = '0123456789abcdefghijklmnopqrstuvwxyz,._'
    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    # integer encode input data
    integer_encoded = [char_to_int[char] for char in dataAdjust]
    #print(integer_encoded)
    # one hot encode
    onehot_encoded = list()
    for i, value in enumerate(integer_encoded):
        letter = [0 for _ in range(len(alphabet))]
        letter[value] = 1
        onehot_encoded.append(letter)
    #print(onehot_encoded) # the real encoding
    return onehot_encoded

# takes a .csv filename
def dataPreprocessing(fileName):
    df = pd.read_csv(fileName, header = None)
    
    #prepare the imput data
    xString = df.iloc[:,:41].to_string(header=False, index=False, index_names = False).split('\n')
    xList = [','.join(ele.split()) for ele in xString] # gives comma separated strings for each row of DataFrame
    xData = []
    for string in xList:
        stringLower = string.lower()
        oneHot = customOneHotEncoder(stringLower)
        xData.append(oneHot)
    xMid = np.array(xData)
    xArray = xMid.transpose(0,2,1) # convert xMid's dim (size, 200, 39) to (size, 39, 200)
    
    #prepare the label data
    df[41] = np.where(df[41]=='normal', 'normal', 'attack') # replacing anything except 'normal' with 'attack'
    Ydf = df[41]
    #labelName = Ydf.unique().tolist().sort() # sorted 38 label names
    #yArray = Ydf.str.get_dummies().to_numpy() # ndarray of shape(rows/lines, 38)
    yArray = Ydf.to_numpy()
    
    assert xArray.shape[0] == yArray.shape[0], 'unequal input and label sample size'
    
    
    return xArray, yArray # return processed array of input and label

In [96]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

class NSLKDDDataset(Dataset):
    def __init__(self, fileName):
        self.data = pd.read_csv(fileName, header = None)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # prepare x data
        string = list(','.join('%s' %x for x in y) for y in self.data.iloc[[idx], :41].values)
        stringLower = string[0].lower()
        xData = customOneHotEncoder(stringLower) # Dim (200, 39)
        xMid = np.array(xData)
        xArray = xMid.transpose() # should be now (39, 200)
        
        # prepate y data
        #self.data.iloc[idx, 41] = np.where(self.data.iloc[idx, 41]=='normal', 0, 1) # replacing normals with 0 and anything else with 1
        yArray = np.where(self.data.iloc[idx, 41]=='normal', 0, 1)
        
        #yArray = Ydf.to_numpy()
    
        #assert xArray.shape == yArray.shape, 'unequal input and label sample size'
        
        return torch.from_numpy(xArray), torch.from_numpy(yArray) # returns torch tensor of x and y

In [118]:
params = {'batch_size': 64, 'shuffle': True}
fileNameTrain = 'KDDTrain+.csv'
fileNameTest = 'KDDTest+.csv'
datasetTrain = NSLKDDDataset(fileNameTrain)
datasetTest = NSLKDDDataset(fileNameTest)
dataGeneratorTrain = DataLoader(datasetTrain, **params)
dataGeneratorTest = DataLoader(datasetTest, **params)


In [119]:
for data, target in dataGeneratorTrain:
    print(target)

tensor([0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0], dtype=torch.int32)
tensor([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1,
        1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1], dtype=torch.int32)
tensor([1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1], dtype=torch.int32)
tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0], dtype=torch.int32)
tensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 

KeyboardInterrupt: 

In [120]:
root = './data/mnist'
batch_size = 64
n_classes = 2
input_channels = 39
seq_length = int(200)
epochs = 100
steps = 0

In [121]:
#train_loader, test_loader = data_generator(root, batch_size)

permute = torch.Tensor(np.random.permutation(784).astype(np.float64)).long()
channel_sizes = [32] * 6#hidden nodes times levels 
kernel_size = 5
model = TCN(input_channels, n_classes, channel_sizes, kernel_size=kernel_size, dropout=0.05)


lr = 1e-5
optimizer = getattr(optim, 'Adam')(model.parameters(), lr=lr)

In [122]:
def train(ep):
    global steps
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(dataGeneratorTrain):
        # print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        optimizer.zero_grad()
        data = data.view(-1, input_channels, seq_length)
        data, target = Variable(data), Variable(target)
        #print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        #print(target)
        optimizer.zero_grad()
        #print(data[0])
        data = data.type(torch.FloatTensor)
        output = model(data)
        #print(output.shape)
        target = target.type(torch.LongTensor)
        #loss1 = torch.nn.CrossEntropyLoss()
        loss = F.nll_loss(output, target) # negative log likelihood
        #loss = loss1(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss
        steps += seq_length
        if batch_idx > 0 and batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tSteps: {}'.format(
                ep, batch_idx * batch_size, len(dataGeneratorTrain.dataset),
                100. * batch_idx / len(dataGeneratorTrain), train_loss.item()/100, steps))
            train_loss = 0

In [123]:
def test():
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorTest:
            model.eval()
            #data = data.view(-1, input_channels, seq_length)
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            #loss1 = torch.nn.CrossEntropyLoss()
            test_loss += F.nll_loss(output, target, size_average=False).item()
            #test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(dataGeneratorTest.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataGeneratorTest.dataset),
            100. * correct / len(dataGeneratorTest.dataset)))
        return test_loss

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (


In [124]:
if __name__ == "__main__":
    for epoch in range(1, epochs+1):
        train(epoch)
        test()
        if epoch % 10 == 0:
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

Train Epoch: 1 [6400/32259 (20%)]	Loss: 0.700584	Steps: 20200
Train Epoch: 1 [12800/32259 (40%)]	Loss: 0.691943	Steps: 40200
Train Epoch: 1 [19200/32259 (59%)]	Loss: 0.691242	Steps: 60200
Train Epoch: 1 [25600/32259 (79%)]	Loss: 0.690697	Steps: 80200
Train Epoch: 1 [32000/32259 (99%)]	Loss: 0.689893	Steps: 100200


C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.



Test set: Average loss: 0.7025, Accuracy: 3594/8496 (42%)

Train Epoch: 2 [6400/32259 (20%)]	Loss: 0.695948	Steps: 121200
Train Epoch: 2 [12800/32259 (40%)]	Loss: 0.688598	Steps: 141200
Train Epoch: 2 [19200/32259 (59%)]	Loss: 0.687412	Steps: 161200
Train Epoch: 2 [25600/32259 (79%)]	Loss: 0.686238	Steps: 181200
Train Epoch: 2 [32000/32259 (99%)]	Loss: 0.684295	Steps: 201200

Test set: Average loss: 0.6961, Accuracy: 3594/8496 (42%)

Train Epoch: 3 [6400/32259 (20%)]	Loss: 0.687317	Steps: 222200
Train Epoch: 3 [12800/32259 (40%)]	Loss: 0.676004	Steps: 242200
Train Epoch: 3 [19200/32259 (59%)]	Loss: 0.668682	Steps: 262200
Train Epoch: 3 [25600/32259 (79%)]	Loss: 0.658987	Steps: 282200
Train Epoch: 3 [32000/32259 (99%)]	Loss: 0.639582	Steps: 302200

Test set: Average loss: 0.6500, Accuracy: 5879/8496 (69%)

Train Epoch: 4 [6400/32259 (20%)]	Loss: 0.602722	Steps: 323200
Train Epoch: 4 [12800/32259 (40%)]	Loss: 0.515981	Steps: 343200
Train Epoch: 4 [19200/32259 (59%)]	Loss: 0.381276	Steps

Train Epoch: 23 [12800/32259 (40%)]	Loss: 0.072961	Steps: 2262200
Train Epoch: 23 [19200/32259 (59%)]	Loss: 0.078941	Steps: 2282200
Train Epoch: 23 [25600/32259 (79%)]	Loss: 0.085217	Steps: 2302200
Train Epoch: 23 [32000/32259 (99%)]	Loss: 0.082463	Steps: 2322200

Test set: Average loss: 0.6739, Accuracy: 6810/8496 (80%)

Train Epoch: 24 [6400/32259 (20%)]	Loss: 0.075091	Steps: 2343200
Train Epoch: 24 [12800/32259 (40%)]	Loss: 0.078921	Steps: 2363200
Train Epoch: 24 [19200/32259 (59%)]	Loss: 0.078070	Steps: 2383200
Train Epoch: 24 [25600/32259 (79%)]	Loss: 0.075506	Steps: 2403200
Train Epoch: 24 [32000/32259 (99%)]	Loss: 0.088634	Steps: 2423200

Test set: Average loss: 0.6738, Accuracy: 6811/8496 (80%)

Train Epoch: 25 [6400/32259 (20%)]	Loss: 0.086755	Steps: 2444200
Train Epoch: 25 [12800/32259 (40%)]	Loss: 0.079543	Steps: 2464200
Train Epoch: 25 [19200/32259 (59%)]	Loss: 0.084194	Steps: 2484200
Train Epoch: 25 [25600/32259 (79%)]	Loss: 0.080415	Steps: 2504200
Train Epoch: 25 [32000/3

Train Epoch: 44 [19200/32259 (59%)]	Loss: 0.076116	Steps: 4403200
Train Epoch: 44 [25600/32259 (79%)]	Loss: 0.084593	Steps: 4423200
Train Epoch: 44 [32000/32259 (99%)]	Loss: 0.075826	Steps: 4443200

Test set: Average loss: 0.6745, Accuracy: 6826/8496 (80%)

Train Epoch: 45 [6400/32259 (20%)]	Loss: 0.081353	Steps: 4464200
Train Epoch: 45 [12800/32259 (40%)]	Loss: 0.075119	Steps: 4484200
Train Epoch: 45 [19200/32259 (59%)]	Loss: 0.080480	Steps: 4504200
Train Epoch: 45 [25600/32259 (79%)]	Loss: 0.080675	Steps: 4524200
Train Epoch: 45 [32000/32259 (99%)]	Loss: 0.080567	Steps: 4544200

Test set: Average loss: 0.6745, Accuracy: 6826/8496 (80%)

Train Epoch: 46 [6400/32259 (20%)]	Loss: 0.074213	Steps: 4565200
Train Epoch: 46 [12800/32259 (40%)]	Loss: 0.082948	Steps: 4585200
Train Epoch: 46 [19200/32259 (59%)]	Loss: 0.077814	Steps: 4605200
Train Epoch: 46 [25600/32259 (79%)]	Loss: 0.078588	Steps: 4625200
Train Epoch: 46 [32000/32259 (99%)]	Loss: 0.077402	Steps: 4645200

Test set: Average loss:

Train Epoch: 65 [25600/32259 (79%)]	Loss: 0.078826	Steps: 6544200
Train Epoch: 65 [32000/32259 (99%)]	Loss: 0.080206	Steps: 6564200

Test set: Average loss: 0.6745, Accuracy: 6826/8496 (80%)

Train Epoch: 66 [6400/32259 (20%)]	Loss: 0.088441	Steps: 6585200
Train Epoch: 66 [12800/32259 (40%)]	Loss: 0.080270	Steps: 6605200
Train Epoch: 66 [19200/32259 (59%)]	Loss: 0.081288	Steps: 6625200
Train Epoch: 66 [25600/32259 (79%)]	Loss: 0.074396	Steps: 6645200
Train Epoch: 66 [32000/32259 (99%)]	Loss: 0.080325	Steps: 6665200

Test set: Average loss: 0.6745, Accuracy: 6826/8496 (80%)

Train Epoch: 67 [6400/32259 (20%)]	Loss: 0.078715	Steps: 6686200
Train Epoch: 67 [12800/32259 (40%)]	Loss: 0.088582	Steps: 6706200
Train Epoch: 67 [19200/32259 (59%)]	Loss: 0.078020	Steps: 6726200
Train Epoch: 67 [25600/32259 (79%)]	Loss: 0.080951	Steps: 6746200
Train Epoch: 67 [32000/32259 (99%)]	Loss: 0.080316	Steps: 6766200

Test set: Average loss: 0.6745, Accuracy: 6826/8496 (80%)

Train Epoch: 68 [6400/32259 (2

Train Epoch: 86 [32000/32259 (99%)]	Loss: 0.077815	Steps: 8685200

Test set: Average loss: 0.6745, Accuracy: 6826/8496 (80%)

Train Epoch: 87 [6400/32259 (20%)]	Loss: 0.080358	Steps: 8706200
Train Epoch: 87 [12800/32259 (40%)]	Loss: 0.089921	Steps: 8726200
Train Epoch: 87 [19200/32259 (59%)]	Loss: 0.080944	Steps: 8746200
Train Epoch: 87 [25600/32259 (79%)]	Loss: 0.074864	Steps: 8766200
Train Epoch: 87 [32000/32259 (99%)]	Loss: 0.076406	Steps: 8786200

Test set: Average loss: 0.6745, Accuracy: 6826/8496 (80%)

Train Epoch: 88 [6400/32259 (20%)]	Loss: 0.085651	Steps: 8807200
Train Epoch: 88 [12800/32259 (40%)]	Loss: 0.083505	Steps: 8827200
Train Epoch: 88 [19200/32259 (59%)]	Loss: 0.083200	Steps: 8847200
Train Epoch: 88 [25600/32259 (79%)]	Loss: 0.079973	Steps: 8867200
Train Epoch: 88 [32000/32259 (99%)]	Loss: 0.064509	Steps: 8887200

Test set: Average loss: 0.6745, Accuracy: 6826/8496 (80%)

Train Epoch: 89 [6400/32259 (20%)]	Loss: 0.075787	Steps: 8908200
Train Epoch: 89 [12800/32259 (4

In [125]:
from sklearn.metrics import classification_report
def get_metrics(dataLoader):
    total = 0
    correct = 0
    precision = 0
    recall = 0
    f1_score = 0
    accuracy = 0
    with torch.no_grad():
        for data, target in dataLoader: # just 1 batch
            model.eval()
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            #print(predicted)
            total+=target.size(0)
            correct+=(predicted == target).sum().item()
            report = classification_report(target, predicted, output_dict=True)
            precision += report['macro avg']['precision']
            recall += report['macro avg']['recall']
            f1_score += report['macro avg']['f1-score']
            accuracy += report['accuracy']
            print(report)
    print("Precision: {}, Recall: {}, F1-Score: {}, Accuracy: {}, AccuracyCust: {}".format(precision, recall, f1_score, accuracy, correct/total))
    

In [126]:
fileNameVal = 'Ds.csv'
datasetVal = NSLKDDDataset(fileNameVal)
params = {'batch_size': 22544, 'shuffle': True}
dataGeneratorVal = DataLoader(datasetVal, **params)
#RuntimeError: expected scalar type Int but found Float
#get_metrics(dataGeneratorVal)
def val():
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorVal:
            model.eval()
            #data = data.view(-1, input_channels, seq_length)
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            #loss1 = torch.nn.CrossEntropyLoss()
            #test_loss += F.nll_loss(output, target, size_average=False).item()
            #test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            print(pred)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(dataGeneratorTest.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataGeneratorVal.dataset),
            100. * correct / len(dataGeneratorVal.dataset)))
        return test_loss

val()


C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\ipykernel_launcher.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  app.launch_new_instance()


tensor([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [1]])

Test set: Average loss: 0.0000, Accuracy: 18134/22544 (80%)



0.0

In [127]:
get_metrics(dataGeneratorVal)

{'0': {'precision': 0.7125671665731013, 'recall': 0.9149418185562764, 'f1-score': 0.8011722272317402, 'support': 9711}, '1': {'precision': 0.918014888337469, 'recall': 0.7207200187017845, 'f1-score': 0.8074908328968047, 'support': 12833}, 'accuracy': 0.8043825408090844, 'macro avg': {'precision': 0.8152910274552851, 'recall': 0.8178309186290305, 'f1-score': 0.8043315300642724, 'support': 22544}, 'weighted avg': {'precision': 0.829516714719044, 'recall': 0.8043825408090844, 'f1-score': 0.8047690452986215, 'support': 22544}}
Precision: 0.8152910274552851, Recall: 0.8178309186290305, F1-Score: 0.8043315300642724, Accuracy: 0.8043825408090844, AccuracyCust: 0.8043825408090844


In [128]:
get_metrics(dataGeneratorTrain)

{'0': {'precision': 0.967741935483871, 'recall': 0.9090909090909091, 'f1-score': 0.9374999999999999, 'support': 33}, '1': {'precision': 0.9090909090909091, 'recall': 0.967741935483871, 'f1-score': 0.9374999999999999, 'support': 31}, 'accuracy': 0.9375, 'macro avg': {'precision': 0.9384164222873901, 'recall': 0.9384164222873901, 'f1-score': 0.9374999999999999, 'support': 64}, 'weighted avg': {'precision': 0.9393328445747801, 'recall': 0.9375, 'f1-score': 0.9374999999999999, 'support': 64}}
{'0': {'precision': 0.9428571428571428, 'recall': 1.0, 'f1-score': 0.9705882352941176, 'support': 33}, '1': {'precision': 1.0, 'recall': 0.9354838709677419, 'f1-score': 0.9666666666666666, 'support': 31}, 'accuracy': 0.96875, 'macro avg': {'precision': 0.9714285714285714, 'recall': 0.967741935483871, 'f1-score': 0.968627450980392, 'support': 64}, 'weighted avg': {'precision': 0.9705357142857143, 'recall': 0.96875, 'f1-score': 0.9686887254901961, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 0.969

{'0': {'precision': 0.9333333333333333, 'recall': 0.9655172413793104, 'f1-score': 0.9491525423728815, 'support': 29}, '1': {'precision': 0.9705882352941176, 'recall': 0.9428571428571428, 'f1-score': 0.9565217391304348, 'support': 35}, 'accuracy': 0.953125, 'macro avg': {'precision': 0.9519607843137254, 'recall': 0.9541871921182266, 'f1-score': 0.9528371407516582, 'support': 64}, 'weighted avg': {'precision': 0.9537071078431372, 'recall': 0.953125, 'f1-score': 0.9531825718496685, 'support': 64}}
{'0': {'precision': 0.9666666666666667, 'recall': 0.9666666666666667, 'f1-score': 0.9666666666666667, 'support': 30}, '1': {'precision': 0.9705882352941176, 'recall': 0.9705882352941176, 'f1-score': 0.9705882352941176, 'support': 34}, 'accuracy': 0.96875, 'macro avg': {'precision': 0.9686274509803922, 'recall': 0.9686274509803922, 'f1-score': 0.9686274509803922, 'support': 64}, 'weighted avg': {'precision': 0.96875, 'recall': 0.96875, 'f1-score': 0.96875, 'support': 64}}
{'0': {'precision': 1.0,

{'0': {'precision': 1.0, 'recall': 0.9523809523809523, 'f1-score': 0.975609756097561, 'support': 42}, '1': {'precision': 0.9166666666666666, 'recall': 1.0, 'f1-score': 0.9565217391304348, 'support': 22}, 'accuracy': 0.96875, 'macro avg': {'precision': 0.9583333333333333, 'recall': 0.9761904761904762, 'f1-score': 0.9660657476139979, 'support': 64}, 'weighted avg': {'precision': 0.9713541666666666, 'recall': 0.96875, 'f1-score': 0.9690482502651114, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 0.9642857142857143, 'f1-score': 0.9818181818181818, 'support': 28}, '1': {'precision': 0.972972972972973, 'recall': 1.0, 'f1-score': 0.9863013698630138, 'support': 36}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.9864864864864865, 'recall': 0.9821428571428572, 'f1-score': 0.9840597758405978, 'support': 64}, 'weighted avg': {'precision': 0.9847972972972974, 'recall': 0.984375, 'f1-score': 0.9843399750933998, 'support': 64}}
{'0': {'precision': 0.9459459459459459, 'recall': 0.972222222222

{'0': {'precision': 0.9393939393939394, 'recall': 1.0, 'f1-score': 0.96875, 'support': 31}, '1': {'precision': 1.0, 'recall': 0.9393939393939394, 'f1-score': 0.96875, 'support': 33}, 'accuracy': 0.96875, 'macro avg': {'precision': 0.9696969696969697, 'recall': 0.9696969696969697, 'f1-score': 0.96875, 'support': 64}, 'weighted avg': {'precision': 0.9706439393939394, 'recall': 0.96875, 'f1-score': 0.96875, 'support': 64}}
{'0': {'precision': 0.972972972972973, 'recall': 1.0, 'f1-score': 0.9863013698630138, 'support': 36}, '1': {'precision': 1.0, 'recall': 0.9642857142857143, 'f1-score': 0.9818181818181818, 'support': 28}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.9864864864864865, 'recall': 0.9821428571428572, 'f1-score': 0.9840597758405978, 'support': 64}, 'weighted avg': {'precision': 0.9847972972972974, 'recall': 0.984375, 'f1-score': 0.9843399750933998, 'support': 64}}
{'0': {'precision': 0.9714285714285714, 'recall': 0.9444444444444444, 'f1-score': 0.9577464788732395, 'supp

{'0': {'precision': 0.9722222222222222, 'recall': 0.9210526315789473, 'f1-score': 0.9459459459459458, 'support': 38}, '1': {'precision': 0.8928571428571429, 'recall': 0.9615384615384616, 'f1-score': 0.9259259259259259, 'support': 26}, 'accuracy': 0.9375, 'macro avg': {'precision': 0.9325396825396826, 'recall': 0.9412955465587045, 'f1-score': 0.9359359359359358, 'support': 64}, 'weighted avg': {'precision': 0.9399801587301587, 'recall': 0.9375, 'f1-score': 0.9378128128128127, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 0.9761904761904762, 'f1-score': 0.9879518072289156, 'support': 42}, '1': {'precision': 0.9565217391304348, 'recall': 1.0, 'f1-score': 0.9777777777777777, 'support': 22}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.9782608695652174, 'recall': 0.9880952380952381, 'f1-score': 0.9828647925033467, 'support': 64}, 'weighted avg': {'precision': 0.9850543478260869, 'recall': 0.984375, 'f1-score': 0.9844544846050869, 'support': 64}}
{'0': {'precision': 1.0, 'recall':

...

{'0': {'precision': 0.9696969696969697, 'recall': 0.9411764705882353, 'f1-score': 0.955223880597015, 'support': 34}, '1': {'precision': 0.9354838709677419, 'recall': 0.9666666666666667, 'f1-score': 0.9508196721311476, 'support': 30}, 'accuracy': 0.953125, 'macro avg': {'precision': 0.9525904203323559, 'recall': 0.9539215686274509, 'f1-score': 0.9530217763640814, 'support': 64}, 'weighted avg': {'precision': 0.9536595796676441, 'recall': 0.953125, 'f1-score': 0.9531594078786396, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 0.918918918918919, 'f1-score': 0.9577464788732395, 'support': 37}, '1': {'precision': 0.9, 'recall': 1.0, 'f1-score': 0.9473684210526316, 'support': 27}, 'accuracy': 0.953125, 'macro avg': {'precision': 0.95, 'recall': 0.9594594594594594, 'f1-score': 0.9525574499629356, 'support': 64}, 'weighted avg': {'precision': 0.9578125, 'recall': 0.953125, 'f1-score': 0.9533682357301706, 'support': 64}}
{'0': {'precision': 0.9722222222222222, 'recall': 0.9722222222222222, 

{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 33}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 31}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}}
{'0': {'precision': 0.9743589743589743, 'recall': 1.0, 'f1-score': 0.9870129870129869, 'support': 38}, '1': {'precision': 1.0, 'recall': 0.9615384615384616, 'f1-score': 0.9803921568627451, 'support': 26}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.9871794871794872, 'recall': 0.9807692307692308, 'f1-score': 0.983702571937866, 'support': 64}, 'weighted avg': {'precision': 0.984775641025641, 'recall': 0.984375, 'f1-score': 0.984323274764451, 'support': 64}}
{'0': {'precision': 0.9696969696969697, 'recall': 0.9696969696969697, 'f1-score': 0.9696969696969697, 'support': 33}, '1': {'precision': 0.967741935483871, 'recall': 0.967741935483871, 'f1-score': 0.967741935

{'0': {'precision': 0.9411764705882353, 'recall': 1.0, 'f1-score': 0.9696969696969697, 'support': 32}, '1': {'precision': 1.0, 'recall': 0.9375, 'f1-score': 0.967741935483871, 'support': 32}, 'accuracy': 0.96875, 'macro avg': {'precision': 0.9705882352941176, 'recall': 0.96875, 'f1-score': 0.9687194525904204, 'support': 64}, 'weighted avg': {'precision': 0.9705882352941176, 'recall': 0.96875, 'f1-score': 0.9687194525904204, 'support': 64}}
{'0': {'precision': 0.9696969696969697, 'recall': 0.9696969696969697, 'f1-score': 0.9696969696969697, 'support': 33}, '1': {'precision': 0.967741935483871, 'recall': 0.967741935483871, 'f1-score': 0.967741935483871, 'support': 31}, 'accuracy': 0.96875, 'macro avg': {'precision': 0.9687194525904204, 'recall': 0.9687194525904204, 'f1-score': 0.9687194525904204, 'support': 64}, 'weighted avg': {'precision': 0.96875, 'recall': 0.96875, 'f1-score': 0.96875, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 0.972972972972973, 'f1-score': 0.986301369863013

{'0': {'precision': 0.9761904761904762, 'recall': 0.9761904761904762, 'f1-score': 0.9761904761904762, 'support': 42}, '1': {'precision': 0.9545454545454546, 'recall': 0.9545454545454546, 'f1-score': 0.9545454545454546, 'support': 22}, 'accuracy': 0.96875, 'macro avg': {'precision': 0.9653679653679654, 'recall': 0.9653679653679654, 'f1-score': 0.9653679653679654, 'support': 64}, 'weighted avg': {'precision': 0.96875, 'recall': 0.96875, 'f1-score': 0.96875, 'support': 64}}
{'0': {'precision': 0.975, 'recall': 0.975, 'f1-score': 0.975, 'support': 40}, '1': {'precision': 0.9583333333333334, 'recall': 0.9583333333333334, 'f1-score': 0.9583333333333334, 'support': 24}, 'accuracy': 0.96875, 'macro avg': {'precision': 0.9666666666666667, 'recall': 0.9666666666666667, 'f1-score': 0.9666666666666667, 'support': 64}, 'weighted avg': {'precision': 0.96875, 'recall': 0.96875, 'f1-score': 0.96875, 'support': 64}}
{'0': {'precision': 0.9354838709677419, 'recall': 0.90625, 'f1-score': 0.92063492063492

{'0': {'precision': 0.9696969696969697, 'recall': 0.9696969696969697, 'f1-score': 0.9696969696969697, 'support': 33}, '1': {'precision': 0.967741935483871, 'recall': 0.967741935483871, 'f1-score': 0.967741935483871, 'support': 31}, 'accuracy': 0.96875, 'macro avg': {'precision': 0.9687194525904204, 'recall': 0.9687194525904204, 'f1-score': 0.9687194525904204, 'support': 64}, 'weighted avg': {'precision': 0.96875, 'recall': 0.96875, 'f1-score': 0.96875, 'support': 64}}
{'0': {'precision': 0.9705882352941176, 'recall': 1.0, 'f1-score': 0.9850746268656716, 'support': 33}, '1': {'precision': 1.0, 'recall': 0.967741935483871, 'f1-score': 0.9836065573770492, 'support': 31}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.9852941176470589, 'recall': 0.9838709677419355, 'f1-score': 0.9843405921213604, 'support': 64}, 'weighted avg': {'precision': 0.9848345588235294, 'recall': 0.984375, 'f1-score': 0.9843635307071201, 'support': 64}}
{'0': {'precision': 0.9714285714285714, 'recall': 0.971428